Dataset : **E-SNLI**. \
Model : **Small T5**.

In [1]:
colab = True

In [2]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt

Cloning into 'ExplainingExplanations'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 111 (delta 54), reused 68 (delta 27), pack-reused 0
Receiving objects: 100% (111/111), 13.51 MiB | 15.18 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/ExplainingExplanations
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 k

# 1.0 Preparation


## 1.1 Loading Dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("esnli")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

Shape of training_set:  (549367, 6)
Shae of validation_set:  (9842, 6)
Shape of test_set:  (9824, 6)


In [6]:
training_set[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

In [7]:
n_train = n_valid = n_test = 500

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

Shape of train_small:  (500, 6)
Shape of valid_small:  (500, 6)
Shape of test_small:  (500, 6)


## 1.2 Loading T5 Model

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Test **zero-shot** on a random task.

In [9]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=100))

Bonjour Dre, je pense que la version anglaise est bonne pour nous.


## 1.3 Zero-shot example to Verify Everything is Working

In [10]:
from src.utils import generate_prompt_mnli

In [11]:
example = training_set[0]
example

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1,
 'explanation_1': 'the person is not necessarily training his horse',
 'explanation_2': '',
 'explanation_3': ''}

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [12]:
prompt = generate_prompt_mnli(example)
prompt

'mnli hypothesis: A person is training his horse for a competition. premise: A person on a horse jumps over a broken down airplane.'

In [13]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

neutral


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## 1.4  Tokenize the dataset

In [14]:
train_small.info

DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None), 'explanation_1': Value(dtype='string', id=None), 'explanation_2': Value(dtype='string', id=None), 'explanation_3': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='esnli', config_name='plain_text', version=0.0.2, splits={'train': SplitInfo(name='train', num_bytes=107611996, num_examples=549367, shard_lengths=None, dataset_name='esnli'), 'validation': SplitInfo(name='validation', num_bytes=3416319, num_examples=9842, shard_lengths=None, dataset_name='esnli'), 'test': SplitInfo(name='test', num_bytes=3379781, num_examples=9824, shard_lengths=None, dataset_name='esnli')}, download_checksums={'hf://datasets/esnli@f01d1430c0ad65a3785d06079a0e01e015e769b8/plain_text/

In [15]:
train_small.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None)}

In [16]:
from functools import partial
from src.utils import tokenize_function
import time

In [17]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [18]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch")
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch")
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch")
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Time taken to tokenize:  1.3502025604248047
Shape of train_small_tokenized:  (500, 9)
Shape of valid_small_tokenized:  (500, 9)
Shape of test_small_tokenized:  (500, 9)


In [19]:
train_small_tokenized.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'explanation_1': Value(dtype='string', id=None),
 'explanation_2': Value(dtype='string', id=None),
 'explanation_3': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

# 2.0 Task 1: Zero-shot evaluation

In [20]:
from src.utils import evaluate_output_mnli
from tqdm import tqdm
import torch

In [21]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

device(type='cuda')

In [22]:
correct = 0
seen = 0
batch_size = 256 # Choose an appropriate batch size

for i in tqdm(range(0, len(train_small_tokenized), batch_size)):
    batch_dict = train_small_tokenized[i : i + batch_size]

    # Prepare input_ids tensor for the entire batch
    input_ids = batch_dict["input_ids"].to(device)

    # Generate outputs for the batch
    outputs = model.generate(input_ids)

    # Iterate through the batch and evaluate each output
    for idx, output_ids in enumerate(outputs):
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        correct += evaluate_output_mnli(output, batch_dict["label"][idx])
        seen += 1

print("Accuracy zero-shot on the test set: ", correct / seen)

100%|██████████| 2/2 [00:06<00:00,  3.01s/it]

Accuracy zero-shot on the test set:  tensor(0.7160)


# 3.0 Task 2: Fine tuning without explanations

In [23]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, AutoTokenizer, T5ForConditionalGeneration

In [24]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-small")

In [25]:
metric = evaluate.load("accuracy")

In [91]:
def compute_metrics(eval_pred, metric):
    label_ids = eval_pred.label_ids
    pred_ids = eval_pred.predictions[0]
    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    print('decoded_preds ', decoded_preds)
    print('decoded_lab ', decoded_labels)
    print(50 * '*')

    return metric.compute(predictions=pred_ids, references= label_ids)

In [92]:
def preprocess_logits_for_metrics(logits, labels):
  pred_ids = torch.argmax(logits[0], dim=-1)
  return pred_ids, labels

In [93]:
compute_accuracy = partial(compute_metrics, metric=metric)

In [94]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=1,
)

In [95]:
trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=test_small_tokenized,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    compute_metrics=compute_accuracy
)

In [96]:
train_small_tokenized["label"].size()

torch.Size([500])

In [97]:
train_small_tokenized["input_ids"].size()

torch.Size([500, 128])

In [98]:
trainer.train()

Epoch,Training Loss,Validation Loss


decoded_preds  ['contra', 'entailment', 'neutralddiction', 'neutral', 'neutralentailment', 'neutralddiction', 'entailment', 'neutral', 'ddiction', 'neutral', 'neutralentailment', 'neutralddiction', 'entailment', 'entailment', 'contraddiction', 'contraddiction', 'entailment', 'neutral', 'entailment', 'contraddiction', 'neutral', 'contraddiction', 'entailment', 'neutral', 'neutral', 'neutralddiction', 'neutralentailment', 'entailment', 'neutral', 'contraddiction', 'contraddiction', 'entailment', 'entailment', 'entailment', 'neutral', 'neutralddiction', '', 'entailment', 'neutralddiction', 'entailment', 'neutral', 'neutralddiction', 'contraddiction', 'neutralentailment', 'neutral', 'neutral', 'contraddiction', 'entailment', 'neutral', 'contraddiction', 'neutral', 'entailment', 'neutralddiction', 'neutral', 'neutralddiction', 'entailment', 'neutralentailment', 'contraddiction', 'neutral', 'entailment', 'ddiction', 'neutral', 'neutral', 'entailment', 'ddiction', 'entailment', 'neutral', 'ne

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: [[ 5314     1     0 ...     0     0     0]
 [    3    35  5756 ...     0     0     0]
 [ 7163    26 12472 ...     0     0     0]
 ...
 [    3    35  5756 ...     0     0     0]
 [ 7163     1     0 ...     0     0     0]
 [ 7163    26 12472 ...     0     0     0]],
Input references: [[ 7163     1     0 ...     0     0     0]
 [    3    35  5756 ...     0     0     0]
 [ 5314    26 12472 ...     0     0     0]
 ...
 [    3    35  5756 ...     0     0     0]
 [ 7163     1     0 ...     0     0     0]
 [ 5314    26 12472 ...     0     0     0]]

In [ ]:
correct = 0
seen = 0
batch_size = 256 # Choose an appropriate batch size
model_ft.to(device)

for i in tqdm(range(0, len(train_small_tokenized), batch_size)):
    batch_dict = train_small_tokenized[i : i + batch_size]

    # Prepare input_ids tensor for the entire batch
    input_ids = batch_dict["input_ids"].to(device)

    # Generate outputs for the batch
    outputs = model_ft.generate(input_ids)

    # Iterate through the batch and evaluate each output
    for idx, output_ids in enumerate(outputs):
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        correct += evaluate_output_mnli(output, batch_dict["label"][idx])
        seen += 1

print("Accuracy zero-shot on the test set: ", correct / seen)

In [ ]:
trainer.evaluate()

# 4.0 Task 4: Making the model generate explanations